In [1]:
import pandas as pd
import json
import numpy as np
import folium
import math

data1 = pd.read_csv('../전처리된 데이터/통합 테이블(공원)_2019.csv')
data2 = pd.read_csv('../전처리된 데이터/통합 테이블(건강)_2019.csv')
data3 = pd.read_csv('../전처리된 데이터/서울시 주민등록인구 (연령별구별) 통계_2019_Q4.csv')
data2.rename(columns={'지역':'자치구'},inplace = True)
data3.set_index('자치구',inplace = True)
data3 = data3[['주민등록인구']]
df2 = data2[data2['연령별'] == '계']
df2 = df2[df2['성별'] == '계']
df2.set_index('자치구',inplace = True)
data1.set_index('자치구',inplace = True)
df = pd.concat([df2,data1,data3],axis=1)
df = df.drop(df.index[0])
df = df.drop(['연령별','성별'],axis=1)

region_data = pd.read_csv('../raw/서울시 행정구역 코드.csv', encoding = 'utf-8')
region_data.set_index('자치구',inplace = True)
total_data = pd.concat([region_data,df],axis=1)
total_data.reset_index(inplace=True)

In [9]:
seoul_geo = '../raw/seoul-maps-master/kostat/2013/json/seoul_municipalities_geo_simple.json'
map_json=open(seoul_geo,encoding='utf-8').read()
seoul_geo_data = json.loads(map_json)

seoul_geo_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'code': '11250',
    'name': '강동구',
    'name_eng': 'Gangdong-gu',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.11519584981606, 37.557533180704915],
      [127.11879551821994, 37.557222485451305],
      [127.12146867175024, 37.55986003393365],
      [127.12435254630417, 37.56144246249796],
      [127.13593925898998, 37.56564793048277],
      [127.14930548011061, 37.56892250303897],
      [127.15511020940411, 37.57093642128295],
      [127.16683184366129, 37.57672487388627],
      [127.17038810813094, 37.576465605301046],
      [127.17607118428914, 37.57678573961056],
      [127.17905504160184, 37.57791388161732],
      [127.17747787800164, 37.57448983055031],
      [127.1781775408844, 37.571481967974336],
      [127.17995281860672, 37.569309661290504],
      [127.18122821955262, 37.56636089217979],
      [127.18169407550688, 37.56286338914073],
      [127.18408792

In [31]:
df_map = pd.DataFrame()
codes = []
regions = []
x_list = []
y_list = []

for i in range(len(seoul_geo_data['features'])):
    code = seoul_geo_data['features'][i]['properties']['code']
    name = seoul_geo_data['features'][i]['properties']['name']
    coordinates = seoul_geo_data['features'][i]['geometry']['coordinates'][0]
    x_max, y_max = 0, 0     # 임의로 설정
    x_min, y_min = 999, 999 # 임의로 설정
    for a in coordinates:
        if a[0] > x_max:
            x_max = a[0]
        if a[0] < x_min:
            x_min = a[0]
        if a[1] > y_max:
            y_max=a[1]
        if a[1] < y_min:
            y_min = a[1]

    codes.append(code)
    regions.append(name)
    
# 이 부분을 통해서 나중에 만들 circle을 최대한 자치구 중심에 넣기 위해서,
# 위도 경도들의 최대값과 최소값의 합을 2로 나눈 위치를 저장해둔다.
    x_list.append((x_max+x_min)/2)
    y_list.append((y_max+y_min)/2)

df_map['Code'] = codes
df_map['자치구'] = regions
df_map['X'] = x_list
df_map['Y'] = y_list

df_circle = pd.merge(total_data,df_map, how = 'left', on = '자치구')

In [50]:
m = folium.Map(location=[37.562225, 126.978555], tiles="OpenStreetMap", zoom_start=11)

m.choropleth(
    geo_data=seoul_geo_data, # json 지도 데이터
    name='1인당 생활권 도시림 면적',
    data=total_data,
    columns=['자치구', '1인당 생활권 도시림 면적'], # data에서 이용할 columns들 설정
    key_on='feature.properties.name', # geo_data에 들어있는 json에서 feature.properties.name와 데이터를 매칭한다
    fill_color='YlGn',
    fill_opacity=0.8,
    line_opacity=0.8,
    color = 'gray',
)


for i in range(0,len(df_circle)):
    location=(df_circle.iloc[i]['Y'], df_circle.iloc[i]['X']) # 경도 위도
    
    color = 'lime'

    m_circle = folium.CircleMarker(location, # 위에서 구한 대략적인 자치구의 중심
                            # 자치구별 상대적인 면적으로 원의 크기 표현
                            radius=(round(df_circle.iloc[i]['1인당 생활권 도시림 면적'])+10)/2,
                            color=color,
                            fill_color=color,
                            fill_opacity = 0.2,
                            opacity=0.2,
                            # popup은 해당 circle을 누르면 나타나는 정보
                            popup=str((round(df_circle.iloc[i]['1인당 생활권 도시림 면적'],2))) + "㎡/인",
                            # tooltip은 마우스를 올려놓으면 나타나는 정보
                            tooltip=df_circle.iloc[i]['자치구'])
    m_circle.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m.save('seoul_forest.html') 
m

In [52]:
data4 = pd.read_csv('../raw/공원위치데이터.csv')
data4 = data4.dropna(axis=0)
data4

,Unnamed: 0,관리번호,공원명,위도,경도,공원면적,자치구
0,150,11680-00061,학동공원,37.511646,127.038764,1498.5,강남구
1,151,11680-00062,신사은행나무공원,37.521114,127.021627,1240.1,강남구
2,152,11680-00063,신사목련공원,37.522898,127.017064,1001.1,강남구
5,155,11680-00066,신사개나리공원,37.526208,127.027304,991.3,강남구
6,156,11680-00067,압구정목련공원,37.532685,127.032209,2027.2,강남구
...,...,...,...,...,...,...,...
1866,17379,11560-00025,중마루근린공원,37.519919,126.911497,6456.0,영등포구
1867,17406,11560-00004,신길근린공원,37.501712,126.905408,19604.0,영등포구
1868,17433,11560-00003,문래근린공원,37.517456,126.894267,23611.0,영등포구
1869,17460,11560-00002,자매근린공원,37.516628,126.929082,16458.0,영등포구


In [54]:
m = folium.Map(location=[data4['위도'].mean(), 
                           data4['경도'].mean()],
                           zoom_start=11, tiles='OpenStreetMap')

m.choropleth(
    geo_data=seoul_geo_data,
    fill_opacity=0,
    line_opacity=0.8,
    fill_color='Greens',
    color = 'gray'
)

for n in data4.index:
    popup_name = data4.loc[n,'공원명']
    
    icon_color = 'green'
    
    folium.CircleMarker(
         location=[data4.loc[n,'위도'],data4.loc[n,'경도']],
         radius =math.sqrt(data4.loc[n,'공원면적'])/100,
         popup = popup_name,
         color=icon_color,
         fill = True,
         fill_color=icon_color     
         
    ).add_to(m) # 위에 변수를 선언하고 사용하는 방법 외에 변수를 사용하지 않고도 사용가능하다

m.save('seoul_park.html')
m